In [1]:
import pandas as pd 
import numpy as np 
import nltk 
nltk.download('wordnet') 
import re
from bs4 import BeautifulSoup 
import contractions
#sklearn
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
 

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vishalvasnani/nltk_data...


In [ ]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## 1. Dataset Preparation: 10 points

Explanation to Dataset Preparation: 1 point

1.1 Read Data : 3 points

## Read Data

In [5]:
#original_df = pd.read_csv('amazon_reviews_us_Kitchen_v1_00.tsv.gz', error_bad_lines = False, warn_bad_lines = False, sep = '\t'
original_df = pd.read_csv('amazon_reviews_us_Beauty_v1_00.tsv.gz',on_bad_lines = 'skip', sep = '\t') 
#df = pd.read_csv('amazon_reviews_us_Kitchen_v1_00.tsv', on_bad_lines = 'skip')error_bad_lines=False

/var/folders/jp/y1xp55js1nqgyd1_ys0_x3t00000gn/T/ipykernel_98440/3376349971.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv('amazon_reviews_us_Beauty_v1_00.tsv.gz',on_bad_lines = 'skip', sep = '\t')


## 1.2 Keep Reviews and Ratings: 3 points

In [7]:
Reviews_and_ratings_cols = [col for col in original_df.columns if ('review_body' in col or 'rating' in col)] 
df = original_df[Reviews_and_ratings_cols]
df = df.dropna()
df = df.reset_index(drop = True)
df.head(3)

,star_rating,review_body
0,5,"Love this, excellent sun block!!"
1,5,The great thing about this cream is that it do...
2,5,"Great Product, I'm 65 years old and this is al..."


 ## 1.3 select 20000 reviews randomly from each rating class: 3 points

In [15]:
class1 = df[(df.star_rating == 1) | (df.star_rating == 2)].sample(20000)
class2 = df[df.star_rating == 3].sample(20000)
class3 = df[(df.star_rating == 4) | (df.star_rating == 5)].sample(20000)

print(' The number of reviews received class 1: ', class1.shape[0]) 
print(' The number of reviews received class 2: ', class2.shape[0]) 
print(' The number of reviews received class 3: ', class3.shape[0])

 The number of reviews received class 1:  20000
 The number of reviews received class 2:  20000
 The number of reviews received class 3:  20000


In [20]:
df_60K = pd.concat([class1 , class2, class3]) 
df_60K = df_60K.reset_index(drop = True)
df_60K.head()

,star_rating,review_body
0,1,Is this really a real product or a fake one?
1,2,got these for my daughters 4th birthday. when ...
2,1,I purchased this product at a local warehouse ...
3,2,"cheap price, cheap quality :/"
4,2,It smells like I remember (good) but the box w...


1.4 train test split

In [21]:
train, test = train_test_split(df_60K, test_size=0.2, random_state = None)
print(len(train))
print(len(test))

48000
12000


# 2. Data Cleaning: 20 points

Explanation to cleaning steps: 5 points

At least one cleaning method must be implemented: 10 points

Convert the all reviews into the lower case.

In [22]:
def lower(texts):
    return texts.lower()

# remove the HTML and URLs from the reviews

Use regular expression <.*?> to exclude all HTML

In [23]:
def cleanhtml(texts):
    #regex = re.compile('<.*?>')
    regex = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(regex, '', texts)
    
    return cleantext

Use regular expression http\S+ to exclude all URLS

In [24]:
def remove_url(texts):
    regex = re.compile('http\S+')
    cleantext = re.sub(regex, '', texts)
    return cleantext

# remove non-alphabetical characters

Use regular expression [^a-zA-Z] to exclude all non alphabetical characters

In [25]:
def non_alphabetical(texts):
    regex = re.compile('[^a-zA-Z]') 
    cleantext = re.sub(regex, ' ', texts)

    regex = re.compile('_')
    cleantext = re.sub(regex, ' ', cleantext)
    
    return cleantext

# Remove the extra spaces between the words

Use regular expression [\s]{2,} to exclude all extra space

In [26]:
def extra_spaces(texts):
    regex = re.compile('[\s]{2,}')
    cleantext = re.sub(regex, ' ', texts)
    
    return cleantext.rstrip()

# perform contractions on the reviews.

use contractions package to contract each sentence

In [27]:
def contractionfunction(text):
    expanded_words = []
    for word in text.split():
        # using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))
        
    expanded_words = ' '.join(expanded_words)
    return expanded_words

In [28]:
def corpus_contractions(texts):
    expended_corpus = []
    for text in texts: 
        expended_corpus.append(contractionfunction(text))
    return expended_corpus

In [29]:
review_body = df_60K.copy(deep = True).review_body.tolist() 
labels = df_60K.copy(deep = True).star_rating.tolist() 
clean_review_body = []

for index , sen in enumerate(review_body):
    sen = lower(sen)
    sen = cleanhtml(sen)
    sen = remove_url(sen)
    sen = contractionfunction(sen)
    sen = non_alphabetical(sen)
    sen = extra_spaces(sen)
    clean_review_body.append(sen)

print the average length of the reviews in terms of character length before Data Cleaning: 2 points

In [30]:
total_len = 0
for sen in review_body:
    #total_len = total_len + len(sen) - sen.count(' ') #after delte space in sentence
    total_len = total_len + len(sen)
print("Average length of reviews before cleaning is :",total_len/len(review_body))

Average length of reviews before cleaning is : 285.84856666666667


print the average length of the reviews in terms of character length after Data Cleaning: 3 points

In [31]:
total_len = 0
for sen in clean_review_body:
    #total_len = total_len + len(sen) - sen.count(' ')
    total_len = total_len + len(sen)
print("Average length of reviews after cleaning is :", total_len/len(clean_review_body))

Average length of reviews after cleaning is : 274.96005


# 3. Pre-processing: 20 points

Comments: 1 point

### 3.1 remove the stop words: 7 points

In [36]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vishalvasnani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishalvasnani/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Apply stopwords.words('english') as a dictionary to exclude all stop words

In [37]:
def remove_stop_words(clean_review_body):
    word_tokens = []
    for sentence in clean_review_body:
        word_tokens.append(word_tokenize(sentence))
    stop_words = set(stopwords.words('english'))
    filtered_reviews_body = []
    for sen in word_tokens:
        filtered_sentence = []
        for w in sen:
            if w not in stop_words: 
                filtered_sentence.append(w)
        new_sentence = " ".join(filtered_sentence) 
        filtered_reviews_body.append(new_sentence)
    return filtered_reviews_body
filtered_reviews_body = remove_stop_words(clean_review_body)

### 3.2 perform lemmatization: 7 points

In [38]:
from nltk.stem import WordNetLemmatizer

In [39]:
def perform_lemmatization(filtered_reviews_body): 
    lemmatizer = WordNetLemmatizer()
    word_tokens = []
    for sentence in filtered_reviews_body:
        word_tokens.append(word_tokenize(sentence))
    lemmatized_reviews_body = []
    for sen in word_tokens: 
        lemmatized_sentence = [] 
        for w in sen:
            new_w = lemmatizer.lemmatize(w) 
            lemmatized_sentence.append(new_w)
        new_sentence = " ".join(lemmatized_sentence) 
        lemmatized_reviews_body.append(new_sentence)
    return lemmatized_reviews_body
lemmatized_reviews_body = perform_lemmatization(filtered_reviews_body)

### print the average length of the reviews in terms of character length before preprocessing: 2 points

In [40]:
total_len = 0
for sen in review_body:
    #total_len = total_len + len(sen) - sen.count(' ')
    total_len = total_len + len(sen)
print("Average length of reviews before cleaning + preprocessing is :",total_len/len(review_body)) 
#review_body[0:3]

Average length of reviews before cleaning + preprocessing is : 285.84856666666667


### print the average length of the reviews in terms of character length after preprocessing: 3 points

In [41]:
total_len = 0
for sen in lemmatized_reviews_body:
    #total_len = total_len + len(sen) - sen.count(' ')
    total_len = total_len + len(sen)
print("Average length of reviews after cleaning + preprocessing is :", total_len/len(lemmatized_reviews_body)) 
#lemmatized_reviews_body[0:3]

Average length of reviews after cleaning + preprocessing is : 164.96575


# 4. TF-IDF Feature Extraction: 10 points

In [42]:
# for every sentence get word by ' '
word_tokens = []
for sentence in lemmatized_reviews_body: 
    for word in sentence.split():
        word_tokens.append(word)

In [43]:
vocabulary = set(word_tokens)
corpus = lemmatized_reviews_body

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names()) 
print(tfidf.shape)

(60000, 32863)


In [44]:
x = tfidf

In [45]:
y = labels

### 5-8: 40 points

Each print out 0.3 points

# 5. Perceptron: 10 points

In [46]:
# Load required libraries
from sklearn import datasets
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [47]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = None)

In [48]:
# Train the scaler, which standarizes all the features to have mean=0 and unit variance
sc = StandardScaler(with_mean=False) 
sc.fit(X_train)

StandardScaler(with_mean=False)

In [49]:
# Apply the scaler to the X training data
X_train_std = sc.transform(X_train)
# Apply the SAME scaler to the X test data
X_test_std = sc.transform(X_test)

In [50]:
# Create a perceptron object with the parameters: 40 iterations (epochs) over the data, and a learning rate of 0.1
ppn = Perceptron(eta0=0.1, random_state=0) 
#ppn.fit(X_train_std, y_train) 
ppn.fit(X_train, y_train)

Perceptron(eta0=0.1)

In [51]:
# Apply the trained perceptron on the X data to make predicts for the y test data 
# y_pred = ppn.predict(X_test_std)
y_pred = ppn.predict(X_test)

In [52]:
precision = precision_score(y_test, y_pred, average=None) 
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

precision_average = precision_score(y_test, y_pred, average="macro") 
recall_average = recall_score(y_test, y_pred, average="macro") 
f1_average = f1_score(y_test, y_pred, average="macro")

In [53]:
print(" Precision Recall F1")
print('Class 1 {:.5f}'.format(precision[0])+', {:.5f}'.format(recall[0])+', {:.5f}'.format(f1[0]))
print('Class 2 {:.5f}'.format(precision[1])+', {:.5f}'.format(recall[1])+', {:.5f}'.format(f1[1]))
print('Class 3 {:.5f}'.format(precision[2])+', {:.5f}'.format(recall[2])+', {:.5f}'.format(f1[2]))
print('Average {:.5f}'.format(precision_average)+', {:.5f}'.format(recall_average)+', {:.5f}'.format(f1_average))

 Precision Recall F1
Class 1 0.50231, 0.52092, 0.51145
Class 2 0.18326, 0.21177, 0.19648
Class 3 0.50309, 0.49268, 0.49783
Average 0.38713, 0.38720, 0.38660


# 6. SVM: 10 points

In [54]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler 
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = None)

In [55]:
#Create a svm Classifier
#clf = make_pipeline(StandardScaler(with_mean=False),LinearSVC(random_state=0, tol=1e-5)) 
clf = LinearSVC()

In [56]:
#Train the model using the training sets
clf.fit(X_train, y_train)

LinearSVC()

In [57]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [58]:
precision = precision_score(y_test, y_pred, average=None) 
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

precision_average = precision_score(y_test, y_pred, average="macro")
recall_average = recall_score(y_test, y_pred, average="macro") 
f1_average = f1_score(y_test, y_pred, average="macro")

In [59]:
print(" Precision Recall F1")
print('Class 1 {:.5f}'.format(precision[0])+', {:.5f}'.format(recall[0])+', {:.5f}'.format(f1[0]))
print('Class 2 {:.5f}'.format(precision[1])+', {:.5f}'.format(recall[1])+', {:.5f}'.format(f1[1]))
print('Class 3 {:.5f}'.format(precision[2])+', {:.5f}'.format(recall[2])+', {:.5f}'.format(f1[2]))
print('Average {:.5f}'.format(precision_average)+', {:.5f}'.format(recall_average)+', {:.5f}'.format(f1_average))

 Precision Recall F1
Class 1 0.58336, 0.65028, 0.61501
Class 2 0.22129, 0.07315, 0.10996
Class 3 0.55066, 0.66873, 0.60398
Average 0.42875, 0.43314, 0.41207


# 7. Logistic Regression: 10 points

In [60]:
from sklearn.linear_model import LogisticRegression

In [61]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = None)

In [62]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

/opt/homebrew/Caskroom/miniforge/base/envs/mlds_hw2/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [63]:
y_pred = clf.predict(X_test)

In [65]:
precision = precision_score(y_test, y_pred, average=None) 
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

precision_average = precision_score(y_test, y_pred, average="macro")
recall_average = recall_score(y_test, y_pred, average="macro") 
f1_average = f1_score(y_test, y_pred, average="macro")

In [66]:
print(" Precision Recall F1")
print('Class 1 {:.5f}'.format(precision[0])+', {:.5f}'.format(recall[0])+', {:.5f}'.format(f1[0]))
print('Class 2 {:.5f}'.format(precision[1])+', {:.5f}'.format(recall[1])+', {:.5f}'.format(f1[1]))
print('Class 3 {:.5f}'.format(precision[2])+', {:.5f}'.format(recall[2])+', {:.5f}'.format(f1[2]))
print('Average {:.5f}'.format(precision_average)+', {:.5f}'.format(recall_average)+', {:.5f}'.format(f1_average))

 Precision Recall F1
Class 1 0.62130, 0.65676, 0.63854
Class 2 0.26744, 0.04662, 0.07940
Class 3 0.54612, 0.72110, 0.62153
Average 0.47820, 0.44539, 0.41698


# 8. Naive Bayes: 10 points

In [67]:
from sklearn.naive_bayes import MultinomialNB

In [68]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = None)

In [69]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [70]:
y_pred = clf.predict(X_test)

In [71]:
precision = precision_score(y_test, y_pred, average=None) 
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

precision_average = precision_score(y_test, y_pred, average="macro") 
recall_average = recall_score(y_test, y_pred, average="macro") 
f1_average = f1_score(y_test, y_pred, average="macro")

/opt/homebrew/Caskroom/miniforge/base/envs/mlds_hw2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/mlds_hw2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [72]:
print(" Precision Recall F1")
print('Class 1 {:.5f}'.format(precision[0])+', {:.5f}'.format(recall[0])+', {:.5f}'.format(f1[0]))
print('Class 2 {:.5f}'.format(precision[1])+', {:.5f}'.format(recall[1])+', {:.5f}'.format(f1[1]))
print('Class 3 {:.5f}'.format(precision[2])+', {:.5f}'.format(recall[2])+', {:.5f}'.format(f1[2]))
print('Average {:.5f}'.format(precision_average)+', {:.5f}'.format(recall_average)+', {:.5f}'.format(f1_average))

 Precision Recall F1
Class 1 0.78905, 0.34043, 0.47565
Class 2 0.00000, 0.00000, 0.00000
Class 3 0.42274, 0.91991, 0.57928
Average 0.39671, 0.36491, 0.34136
